#Flight Delays Data
In this section we’ll walk through a few examples of queries on the Airline On-Time
Performance and Causes of Flight Delays data set, which contains data on US flights
including date, delay, distance, origin, and destination. It’s available as a CSV file with
over a million records.
We will use SQL in your Spark applications via the spark.sql programmatic interface. Similar to the DataFrame API in its declarative
flavor, this interface allows you to query structured data in your Spark
applications.

In [2]:
from pyspark.sql import SparkSession
# Create a SparkSession
spark = (SparkSession
         .builder
         .appName("SparkSQLExampleApp")
         .getOrCreate())

In [3]:
# Path to data set
csv_file = "/databricks-datasets/learning-spark-v2/flights/departuredelays.csv"

# Read and create a temporary view
# Infer schema (note that for larger files you
# may want to specify the schema)
df = (spark.read.format("csv")
      .option("inferSchema", "true")
      .option("header", "true")
      .load(csv_file))

df.createOrReplaceTempView("us_delay_flights_tbl")

In [4]:
df.show()

+-------+-----+--------+------+-----------+
 date|delay|distance|origin|destination|
+-------+-----+--------+------+-----------+
1011245| 6| 602| ABE| ATL|
1020600| -8| 369| ABE| DTW|
1021245| -2| 602| ABE| ATL|
1020605| -4| 602| ABE| ATL|
1031245| -4| 602| ABE| ATL|
1030605| 0| 602| ABE| ATL|
1041243| 10| 602| ABE| ATL|
1040605| 28| 602| ABE| ATL|
1051245| 88| 602| ABE| ATL|
1050605| 9| 602| ABE| ATL|
1061215| -6| 602| ABE| ATL|
1061725| 69| 602| ABE| ATL|
1061230| 0| 369| ABE| DTW|
1060625| -3| 602| ABE| ATL|
1070600| 0| 369| ABE| DTW|
1071725| 0| 602| ABE| ATL|
1071230| 0| 369| ABE| DTW|
1070625| 0| 602| ABE| ATL|
1071219| 0| 569| ABE| ORD|
1080600| 0| 369| ABE| DTW|
+-------+-----+--------+------+-----------+
only showing top 20 rows

Now that we have a temporary view, we can issue SQL queries using Spark SQL.

#Query 1:
Find out all flights whose distance between origin and destination is greater than 1000

In [7]:
(spark.sql("""SELECT distance, origin, destination 
              FROM us_delay_flights_tbl 
              WHERE distance > 1000 
              ORDER BY distance DESC""")
.show(10, truncate=False))

+--------+------+-----------+
distance|origin|destination|
+--------+------+-----------+
4330 |HNL |JFK |
4330 |HNL |JFK |
4330 |HNL |JFK |
4330 |HNL |JFK |
4330 |HNL |JFK |
4330 |HNL |JFK |
4330 |HNL |JFK |
4330 |HNL |JFK |
4330 |HNL |JFK |
4330 |HNL |JFK |
+--------+------+-----------+
only showing top 10 rows

In [8]:
"""Equivalent in DataFrame"""

from pyspark.sql.types import *
from pyspark.sql.functions import *

(df.select("distance", "origin", "destination")
 .where(col("distance") > 1000)
 .orderBy(desc("distance"))
 .show(10, truncate=False))

+--------+------+-----------+
distance|origin|destination|
+--------+------+-----------+
4330 |HNL |JFK |
4330 |HNL |JFK |
4330 |HNL |JFK |
4330 |HNL |JFK |
4330 |HNL |JFK |
4330 |HNL |JFK |
4330 |HNL |JFK |
4330 |HNL |JFK |
4330 |HNL |JFK |
4330 |HNL |JFK |
+--------+------+-----------+
only showing top 10 rows

#Query 2:
Find out all flights with 2 hour delays between San Francisco and Chicago

In [10]:
(spark.sql("""SELECT date, delay, origin, destination 
              FROM us_delay_flights_tbl 
              WHERE delay > 120 AND origin = 'SFO' AND destination = 'ORD'
              ORDER BY delay DESC""")
.show(10, truncate=False))

+-------+-----+------+-----------+
date |delay|origin|destination|
+-------+-----+------+-----------+
2190925|1638 |SFO |ORD |
1031755|396 |SFO |ORD |
1022330|326 |SFO |ORD |
1051205|320 |SFO |ORD |
1190925|297 |SFO |ORD |
2171115|296 |SFO |ORD |
1071040|279 |SFO |ORD |
1051550|274 |SFO |ORD |
3120730|266 |SFO |ORD |
1261104|258 |SFO |ORD |
+-------+-----+------+-----------+
only showing top 10 rows

#Query 3:
A more complicated query in SQL, let's label all US flights originating from airports with high, medium, low, no delays, regardless of destinations.

In [12]:
spark.sql("""SELECT delay, origin, destination,
              CASE
                  WHEN delay > 360 THEN 'Very Long Delays'
                  WHEN delay > 120 AND delay < 360 THEN  'Long Delays '
                  WHEN delay > 60 AND delay < 120 THEN  'Short Delays'
                  WHEN delay > 0 and delay < 60  THEN   'Tolerable Delays'
                  WHEN delay = 0 THEN 'No Delays'
                  ELSE 'No Delays'
               END AS Flight_Delays
               FROM us_delay_flights_tbl
               ORDER BY origin, delay DESC""").show(10, truncate=False)

+-----+------+-----------+-------------+
delay|origin|destination|Flight_Delays|
+-----+------+-----------+-------------+
333 |ABE |ATL |Long Delays |
305 |ABE |ATL |Long Delays |
275 |ABE |ATL |Long Delays |
257 |ABE |ATL |Long Delays |
247 |ABE |DTW |Long Delays |
247 |ABE |ATL |Long Delays |
219 |ABE |ORD |Long Delays |
211 |ABE |ATL |Long Delays |
197 |ABE |DTW |Long Delays |
192 |ABE |ORD |Long Delays |
+-----+------+-----------+-------------+
only showing top 10 rows